# Setting up of training environment

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Dec  6 16:54:11 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0    26W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install datasets transformers==4.9.2 seqeval
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!git clone https://github.com/NVIDIA/apex
%cd apex
!pip install -v --no-cache-dir ./

fatal: destination path 'apex' already exists and is not an empty directory.
/content/apex
Using pip 22.3.1 from /usr/local/lib/python3.8/dist-packages/pip (python 3.8)
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/apex
  Running command python setup.py egg_info


  torch.__version__  = 1.13.0+cu117


  running egg_info
  creating /tmp/pip-pip-egg-info-37m4zxly/apex.egg-info
  writing /tmp/pip-pip-egg-info-37m4zxly/apex.egg-info/PKG-INFO
  writing dependency_links to /tmp/pip-pip-egg-info-37m4zxly/apex.egg-info/dependency_links.txt
  writing requirements to /tmp/pip-pip-egg-info-37m4zxly/apex.egg-info/requires.txt
  writing top-level names to /tmp/pip-pip-egg-info-37m4zxly/apex.egg-info/top_level.txt
  writing manifest file '/tmp/pip-pip-egg-info-37m4zxly/apex.egg-info/SOURCES.txt'
  adding license file 'LICENSE'
  writing manifest file '/tmp/pip-pip-egg-info-37m4zxly/apex.egg-info/SOURCES.txt'
  Preparing metadat

In [ ]:
# !pip install torchtext --no-binary :all:
!pip install torchtext==0.6.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install --upgrade pip setuptools wheel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!apt-get install libhdf5-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libhdf5-dev is already the newest version (1.10.0-patch1+docs-4).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 7 not upgraded.


In [ ]:
# importing torch and setting seed value
import torch
SEED = 1111
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [190]:
import pandas as pd
import numpy as np
import glob
import sys
import os
import re

from transformers import (AutoModel, AutoTokenizer, BertModel, BertPreTrainedModel, AutoModelForSequenceClassification,
                          AutoConfig, AutoModelForTokenClassification, TrainingArguments, 
                          Trainer, BertTokenizer, DataCollatorForTokenClassification)

from datasets import (
    load_dataset, load_metric,
    Dataset,
    DatasetDict,
    Features, Sequence, ClassLabel, Value
)

import torch
import torch.nn as nn
from torch.nn import CrossEntropyLoss, MSELoss

from sklearn.metrics import classification_report

In [192]:
class BertForSequenceClassification(BertPreTrainedModel):
  def __init__(self, config, weight=None):
        super(BertForSequenceClassification, self).__init__(config)
        self.num_labels = config.num_labels

        self.bert_1 = BertModel(config)
        self.bert_2 = BertModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size * 2, self.config.num_labels)
        self.weight = weight

        self.init_weights()

In [193]:
model_name = 'airesearch/wangchanberta-base-att-spm-uncased'
config = AutoConfig.from_pretrained(model_name, num_labels=len(labels))
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, 
                                                           config=config)
len(tokenizer)

Some weights of the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased were not used when initializing CamembertForSequenceClassification: ['lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at airesearch/wa

25005

In [194]:
# an example to see the working of the BERT tokenizer
tokens = tokenizer.tokenize('คุณมีหนังยางรัดผมมั้ย ขอหน่อยสิของผมมันหล่นหายไปตอนนั่งเรือเมื่อกี้น่ะ')
print(tokens)

['▁คุณ', 'มี', 'หนัง', 'ยาง', 'รัด', 'ผม', 'มั้ย', '▁ขอ', 'หน่อยสิ', 'ของผม', 'มัน', 'หล่น', 'หายไป', 'ตอน', 'นั่ง', 'เรือ', 'เมื่อกี้', 'น่ะ']


In [ ]:
# Tokens can be easily converted to index using a BERT tokenizer
indexes = tokenizer.convert_tokens_to_ids(tokens)
print(indexes)

[296, 23, 684, 1158, 5189, 92, 556, 1212, 8302, 1315, 66, 6728, 1711, 223, 468, 547, 8229, 444]


BERT takes all the inputs as a single sequence. If we need to provide more than one input as in our case where our input will be premise and hypothesis, [SEP] token helps the model to understand input properly. Let’s understand the input format with the help of an example.

Premise: Man is wearing blue jeans.<br/>
Hypothesis: Man is wearing red jeans.<br/>
Input Format: [CLS] Man is wearing blue jeans. [SEP] Man is wearing red jeans. [SEP]<br/>

So, here our input to the model will be in the format given above. The pad as well as the unknown token should also match with the BERT tokenizer. We can get these tokens as well as the index corresponding to these tokens from the tokenizer.

In [195]:
cls_token = tokenizer.cls_token
sep_token = tokenizer.sep_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token
print(cls_token, sep_token, pad_token, unk_token)

<s> </s> <pad> <unk>


Output:<br/>
[CLS]   [SEP]   [PAD]   [UNK]

In [196]:
cls_token_idx = tokenizer.cls_token_id
sep_token_idx = tokenizer.sep_token_id
pad_token_idx = tokenizer.pad_token_id
unk_token_idx = tokenizer.unk_token_id
print(cls_token_idx, sep_token_idx, pad_token_idx, unk_token_idx)

5 6 1 3


In [197]:
max_input_length = 256

In [198]:
def tokenize_bert(sentence):
    tokens = tokenizer.tokenize(sentence) 
    return tokens

In [199]:
def split_and_cut(sentence):
    tokens = sentence.strip().split(" ")
    tokens = tokens[:max_input_length]
    return tokens

# Download and Prepare datasets

In [267]:
# train set
!gdown --id 1DxBuSj2YZN_rdRBk5ZGefioB8CN1aqhk

# dev set
!gdown --id 1z1humh123OJO2YUiJQ3A2Ww97f5n8P3F

# test set
!gdown --id 1rBxajNnPSszytIr3fL6CVERkdVmjosgm

/usr/local/lib/python3.8/dist-packages/gdown/cli.py:127: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1DxBuSj2YZN_rdRBk5ZGefioB8CN1aqhk
To: /content/apex/NLP3_Final_Project-Train_Set.csv
100% 71.9k/71.9k [00:00<00:00, 54.3MB/s]
/usr/local/lib/python3.8/dist-packages/gdown/cli.py:127: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1z1humh123OJO2YUiJQ3A2Ww97f5n8P3F
To: /content/apex/NLP3_Final_Project-Dev_Set.csv
100% 9.80k/9.80k [00:00<00:00, 12.0MB/s]
/usr/local/lib/python3.8/dist-packages/gdown/cli.py:127: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
D

Define helper functions for data preparation

In [268]:
#Get list of 0s 
def get_sent1_token_type(sent):
    try:
        return [0]* len(sent)
    except:
        return []

#Get list of 1s
def get_sent2_token_type(sent):
    try:
        return [1]* len(sent)
    except:
        return []

#combine from lists
def combine_seq(seq):
    return " ".join(seq)

#combines from lists of int
def combine_mask(mask):
    mask = [str(m) for m in mask]
    return " ".join(mask)

In [269]:
# import data
df_train = pd.read_csv('NLP3_Final_Project-Train_Set.csv', encoding='utf-8')
df_dev = pd.read_csv('NLP3_Final_Project-Dev_Set.csv', encoding='utf-8')
df_test = pd.read_csv('NLP3_Final_Project-Test_Set.csv', encoding='utf-8')

In [270]:
df_test

,id,question,answer,label
0,271,เธอกับแฟนยังรักกันดีหรือเปล่า,เราเหลือเวลาไม่มาก พ่อจะเช็คตารางเพื่อหาเที่ยว...,0
1,272,เธอคิดว่านิยายเรื่องนี้สนุกไหม,ผัดผักไม่กิน แต่กลับอยากกินผัดผักบุ้งฝีมือแม่ท...,0
2,273,คืนนี้คุณโทรหาผมได้หรือเปล่า,ให้ตายเถอะ เธอทนลูกอ้อนของยัยคนนี้ไม่ได้จริง ๆ,0
3,274,เราไปคอนเสิร์ตของน้าแอ๊ดกันไหม,ฉันสืบประวัติของเขามาหมดแล้วก่อนที่จะกลับมา,0
4,275,ไปอยู่อังกฤษตั้งนาน เธอคิดถึงบ้านบ้างหรือเปล่า,ฉันทำสัญญาแทนนายเรียบร้อยแล้ว,0
5,276,ต้องให้ผมพิสูจน์อีกแค่ไหน คุณถึงจะคิดว่าผมไม่ใ...,ตั้งแต่ตอนนี้เป็นต้นไป คนที่ฉันจะเดินไปส่งที่บ...,0
6,277,ถ้าเป็นไปได้ เธออยากจะขอลายเซ็นดาราคนนั้นอยู่ไหม,ไม่ทันแล้ว มันล้างเลือดของข้าออกหมดแล้ว,0
7,278,เราออกจากฝั่งมาไกลมากแล้ว ว่ายกลับเข้าฝั่งกันเถอะ,ยายเดินกลับมาบ้านแล้ว เสียงก๊อกแก๊ก ๆ อยู่ในคร...,0
8,279,เธอจะถ่ายรูปอยู่ตรงนี้ทั้งวัน ไม่ไปที่อื่นแล้ว...,ครั้งสุดท้ายคุณนายติดตามสามีขึ้นไปรับราชการอยู...,0
9,280,เธอคิดว่าข้อสอบปลายภาควิชานี้ยากไหม,เมื่อหลายชั่วโมงก่อนมันกลับไปที่โบสถ์ แต่ตอนนี...,0


In [271]:
labels = list(set(df_train['label'].to_list()))
label_to_idx = {label:i for i,label in enumerate(labels)}
df_train['label'] = df_train['label'].apply(lambda x: label_to_idx[x])
df_dev['label'] = df_dev['label'].apply(lambda x: label_to_idx[x])
df_test['label'] = df_test['label'].apply(lambda x: label_to_idx[x])

label_list = [0] * len(label_to_idx)
for label in label_to_idx:
  label_list[label_to_idx[label]] = label

In [272]:
label_list
# '0' is NEUTRAL
# '1' is YES
# '2' is NO

[0, 1, 2]

In [297]:
#Add [SEP] tokens
df_train['sent1'] = df_train['question'] + ' </s> '
df_dev['sent1'] = df_dev['question'] + ' </s> '
df_test['sent1'] = df_test['question'] + ' </s> '

In [274]:
#Combine both sequences
df_train['sequence'] = df_train['sent1'] + df_train['answer']
df_dev['sequence'] = df_dev['sent1'] + df_dev['answer']
df_test['sequence'] = df_test['sent1'] + df_test['answer']

In [275]:
df_test['sequence']

0     เธอกับแฟนยังรักกันดีหรือเปล่า </s> เราเหลือเวล...
1     เธอคิดว่านิยายเรื่องนี้สนุกไหม </s> ผัดผักไม่ก...
2     คืนนี้คุณโทรหาผมได้หรือเปล่า </s> ให้ตายเถอะ เ...
3     เราไปคอนเสิร์ตของน้าแอ๊ดกันไหม </s> ฉันสืบประว...
4     ไปอยู่อังกฤษตั้งนาน เธอคิดถึงบ้านบ้างหรือเปล่า...
5     ต้องให้ผมพิสูจน์อีกแค่ไหน คุณถึงจะคิดว่าผมไม่ใ...
6     ถ้าเป็นไปได้ เธออยากจะขอลายเซ็นดาราคนนั้นอยู่ไ...
7     เราออกจากฝั่งมาไกลมากแล้ว ว่ายกลับเข้าฝั่งกันเ...
8     เธอจะถ่ายรูปอยู่ตรงนี้ทั้งวัน ไม่ไปที่อื่นแล้ว...
9     เธอคิดว่าข้อสอบปลายภาควิชานี้ยากไหม </s> เมื่อ...
10    ต้มยำกุ้งหม้อนี้ป้าอรเป็นคนปรุงใช่มั้ย </s> รส...
11    หนังสือถูกฉีกไปหน้านึง เธอเห็นมั้ยว่าใครเป็นคน...
12    เธอจะเอากระเป๋าใบนี้ไปประมูลใช่มั้ย </s> ฉันต้...
13    นี่นายไม่ชอบไก่มากขนาดนั้นเลยเหรอ </s> ทุกวันน...
14    พี่จะด่าหนูมั้ย ถ้าหนูบอกว่าเงินของเดือนนี้หนู...
15    เขาบอกว่าหาดสวยมาก นักท่องเที่ยวส่วนมากมีแต่ฝร...
16    เมื่อกี้ฉันไม่ได้หูฝาดไปใช่มั้ยที่ได้ยินว่ามนุ...
17    คุณช่วยหาที่อยู่ให้เราได้มั้ย ที่ที่ไม่มีใ

In [276]:
features= Features({
                    #"title tokens": Sequence(Value(dtype='string')),
                    "sequence": Value(dtype='string'),
                    
                    "label": ClassLabel(names=label_list)
                })
d = DatasetDict({'train': Dataset.from_pandas(df_train, features=features),
                 'test': Dataset.from_pandas(df_test), 
                 'dev': Dataset.from_pandas(df_dev, features=features)})

In [277]:
d

DatasetDict({
    train: Dataset({
        features: ['sequence', 'label'],
        num_rows: 240
    })
    test: Dataset({
        features: ['id', 'question', 'answer', 'label', 'sent1', 'sequence'],
        num_rows: 30
    })
    dev: Dataset({
        features: ['sequence', 'label'],
        num_rows: 30
    })
})

In [278]:
d['train'][0]

{'sequence': 'น้องนิกอย่าเพิ่งทำอะไรวู่วามนะ เดี๋ยวพี่ทำธุระเสร็จจะแวะไปคุยด้วย ตกลงมั้ย </s> แม่จะกลับมาจากต่างประเทศสิ้นปีนี้',
 'label': 0}

In [279]:
def tokenize(examples):
    tokenized_inputs = tokenizer(examples["sequence"], 
                                 is_split_into_words=False,
                                 truncation=True,
                                 max_length=50)
    return tokenized_inputs
tokenized_datasets = d.map(tokenize, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [280]:
tokenizer.convert_ids_to_tokens(tokenized_datasets['train'][0]['input_ids'])

['<s>',
 '▁น้อง',
 'นิก',
 'อย่าเพิ่ง',
 'ทําอะไร',
 'วู',
 '่ว',
 'า',
 'ม',
 'นะ',
 '▁เดี๋ยว',
 'พี่',
 'ทําธุระ',
 'เสร็จ',
 'จะ',
 'แวะไป',
 'คุยด้วย',
 '▁ตกลง',
 'มั้ย',
 '▁',
 '</s>',
 '▁แม่',
 'จะกลับมา',
 'จาก',
 'ต่างประเทศ',
 'สิ้นปี',
 'นี้',
 '</s>']

In [281]:
model_name.split('/')[-1]

'wangchanberta-base-att-spm-uncased'

In [282]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'label', 'sequence'],
        num_rows: 240
    })
    test: Dataset({
        features: ['answer', 'attention_mask', 'id', 'input_ids', 'label', 'question', 'sent1', 'sequence'],
        num_rows: 30
    })
    dev: Dataset({
        features: ['attention_mask', 'input_ids', 'label', 'sequence'],
        num_rows: 30
    })
})

In [284]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer,
                                        padding=True, pad_to_multiple_of=8)
tokenized_datasets = tokenized_datasets.remove_columns(["sequence"])
#tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
#tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names

['attention_mask', 'input_ids', 'label']

In [285]:
tokenized_datasets['dev'][20]

{'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'input_ids': [5,
  645,
  1477,
  12,
  8502,
  50,
  9,
  79,
  314,
  3370,
  19953,
  10,
  6,
  10,
  15721,
  381,
  78,
  4574,
  12708,
  12,
  346,
  222,
  578,
  12,
  346,
  557,
  393,
  707,
  5499,
  1766,
  6],
 'label': 2}

In [286]:
samples = tokenized_datasets["train"][:8]
samples = {k: v for k, v in samples.items()}
[len(x) for x in samples["input_ids"]]

[28, 32, 30, 30, 31, 27, 25, 28]

In [287]:
batch = data_collator(samples)
{k: v.shape for k, v in batch.items()}

{'attention_mask': torch.Size([8, 32]),
 'input_ids': torch.Size([8, 32]),
 'labels': torch.Size([8])}

In [288]:
from datasets import list_metrics, load_metric
list_metrics()

['accuracy',
 'bertscore',
 'bleu',
 'bleurt',
 'brier_score',
 'cer',
 'chrf',
 'code_eval',
 'comet',
 'competition_math',
 'coval',
 'cuad',
 'exact_match',
 'f1',
 'frugalscore',
 'glue',
 'google_bleu',
 'indic_glue',
 'mae',
 'mahalanobis',
 'mape',
 'mase',
 'matthews_correlation',
 'mauve',
 'mean_iou',
 'meteor',
 'mse',
 'nist_mt',
 'pearsonr',
 'perplexity',
 'poseval',
 'precision',
 'recall',
 'rl_reliability',
 'roc_auc',
 'rouge',
 'sacrebleu',
 'sari',
 'seqeval',
 'smape',
 'spearmanr',
 'squad',
 'squad_v2',
 'super_glue',
 'ter',
 'trec_eval',
 'wer',
 'wiki_split',
 'xnli',
 'xtreme_s',
 'Drunper/metrica_tesi',
 'Felipehonorato/my_metric',
 'GMFTBY/dailydialog_evaluate',
 'GMFTBY/dailydialogevaluate',
 'JP-SystemsX/nDCG',
 'KevinSpaghetti/accuracyk',
 'NCSOFT/harim_plus',
 'NikitaMartynov/spell-check-metric',
 'NimaBoscarino/weat',
 'Ochiroo/rouge_mn',
 'Vertaix/vendiscore',
 'Viona/infolm',
 'Vlasta/pr_auc',
 'abdusah/aradiawer',
 'abidlabs/mean_iou',
 'abidlabs/me

In [289]:
load_metric('accuracy')

Downloading:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

Metric(name: "accuracy", features: {'predictions': Value(dtype='int32', id=None), 'references': Value(dtype='int32', id=None)}, usage: """
Args:
    predictions: Predicted labels, as returned by a model.
    references: Ground truth labels.
    normalize: If False, return the number of correctly classified samples.
        Otherwise, return the fraction of correctly classified samples.
    sample_weight: Sample weights.
Returns:
    accuracy: Accuracy score.
Examples:

    >>> accuracy_metric = datasets.load_metric("accuracy")
    >>> results = accuracy_metric.compute(references=[0, 1], predictions=[0, 1])
    >>> print(results)
    {'accuracy': 1.0}
""", stored examples: 0)

# Training Model

In [353]:
import numpy as np
args = TrainingArguments(
    output_dir='label-pred'
)

# for hyperparameter tuning, please train with 'arg2'
'''args2 = TrainingArguments(
    output_dir='label-pred2',
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=16,
    num_train_epochs=4,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    evaluation_strategy='steps', 
    eval_steps=10,
    #save_strategy='epoch'    
)'''
#metric.compute(predictions=preds, references=predictions.label_ids)

def compute_metrics(eval_preds):
  metric = load_metric("accuracy")
  logits, labels = eval_preds
  predictions = np.argmax(logits, axis=-1)
  return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["dev"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running training *****
  Num examples = 240
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 90


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=90, training_loss=1.0207667032877603, metrics={'train_runtime': 8.4921, 'train_samples_per_second': 84.785, 'train_steps_per_second': 10.598, 'total_flos': 13945011139584.0, 'train_loss': 1.0207667032877603, 'epoch': 3.0})

# Test and Evaluation

In [354]:
predictions = trainer.predict(tokenized_datasets['test'])
preds = np.argmax(predictions.predictions, axis=-1)

The following columns in the test set  don't have a corresponding argument in `CamembertForSequenceClassification.forward` and have been ignored: question, answer, sent1, id.
***** Running Prediction *****
  Num examples = 30
  Batch size = 8


In [355]:
predictions

PredictionOutput(predictions=array([[ 0.70581865,  0.08694888, -0.65737915],
       [ 0.7079466 ,  0.08637337, -0.65926033],
       [ 0.70298237,  0.08768619, -0.654832  ],
       [ 0.70673364,  0.08670763, -0.6581909 ],
       [ 0.70497555,  0.08717377, -0.65663075],
       [ 0.6760335 ,  0.09457607, -0.62982863],
       [ 0.69927496,  0.08867147, -0.65147126],
       [ 0.7078958 ,  0.0863897 , -0.65921414],
       [ 0.70318305,  0.08765764, -0.65504926],
       [ 0.7049595 ,  0.08718777, -0.65662503],
       [ 0.70708793,  0.08660784, -0.6585066 ],
       [ 0.70608103,  0.08688546, -0.6576245 ],
       [ 0.6970221 ,  0.08923206, -0.6493815 ],
       [ 0.7000282 ,  0.0884914 , -0.65218925],
       [ 0.6975816 ,  0.08907009, -0.64985573],
       [ 0.69545716,  0.08966776, -0.64798623],
       [ 0.7080327 ,  0.08635167, -0.65932286],
       [ 0.7019341 ,  0.08797305, -0.65389013],
       [-0.81126785, -0.03170582,  1.764961  ],
       [ 0.7014851 ,  0.08810843, -0.6535103 ],
       [ 0.

In [356]:
predictions.label_ids

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2])

In [357]:
preds

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0])

In [358]:
predicted_labels = [labels[x] for x in preds]
gold = [labels[x] for x in predictions.label_ids]

In [359]:
from sklearn.metrics import classification_report
print(classification_report(gold, predicted_labels))

              precision    recall  f1-score   support

           0       0.34      1.00      0.51        10
           1       0.00      0.00      0.00        10
           2       0.00      0.00      0.00        10

    accuracy                           0.33        30
   macro avg       0.11      0.33      0.17        30
weighted avg       0.11      0.33      0.17        30



/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
